# Link Creation Agent
This notebook demonstrates the use of the **Link Creation Agent (LCA)** in `hyperon-experimental` to generate expressions (links) using pattern-matching queries.

The **DAS Link Creation Agent (LCA)** generates new expressions (links) within a knowledge base by processing queries through **templates** or specialized **processors**. Together, these components define an **LCA Request**.   
An LCA Request can be executed a specified number of times or configured to run indefinitely, with the goal of continuously updating the database with relevant links over time.  

Each execution of an LCA Request is referred to as a **request cycle**. During a cycle, LCA performs a pattern-matching query, retrieves all matching results, and then applies the specified template or processor to transform these query answers into new expressions. The generated links/expressions are inserted back into the knowledge base, potentially influencing subsequent cycles. This iterative process enables the system to dynamically evolve its internal structure as new relationships are discovered and added over time.  

The **query parameters** control the behavior of the query component within the LCA Request. They can define the context, determine how importance values are updated, limit the number of returned query answers, or restrict results based on importance value.  
Additionally, the overall execution behavior of the LCA Request can be configured through parameters such as the **repeat count**, which determines how many times the request should run, an **interval** (in seconds) can also be defined to specify the delay between consecutive executions of the LCA Request.

For full DAS documentation check [here](https://github.com/singnet/das), and more LCA documentation [here](https://github.com/singnet/das/tree/master/src/agents/link_creation_agent)

**Before running this notebook, execute the [setup procedure/notebook](https://github.com/singnet/das/blob/master/notebooks/das-he-setup.ipynb) to load the [animal_dataset](https://github.com/singnet/das/blob/master/src/tests/integration/cpp/data/animal_expressions.metta).**

## Setup
Install the dependencies

In [ ]:
!pip install hyperon

Import necessary modules and initialize Hyperon with DAS knowledge base

In [ ]:
import hyperon

metta = hyperon.MeTTa()
def run(program='!(+ 1 2)'):
    for result in metta.run(program):
        for child in result:
            print(child)

In [ ]:
run('!(import! &self das)')

Bind the Link Creation Agent (LCA) to &das namespace

In [ ]:
run('!(bind! &das (new-das! (localhost:47000-47999) (localhost:31700)))')

## LCA Configuration

Execute the command `!(das-get-params!)` to display the current DAS parameters and view the default settings.  

LCA supports the following **query parameters**, which control the behavior of the query component within an **LCA Request**:  
* **context**: Defines the context in which the query should be executed.  
* **max_answers**: Specifies the maximum number of answers that the query should return.  
* **attention_update_flag**: Determines whether the query should update the importance (attention) values of the retrieved answers.  
* **positive_importance_flag**: When enabled, restricts the query results to answers with an importance value greater than zero.  

In addition, the following parameters are specific to **LCA** and control the overall execution behavior of an **LCA Request**:  
* **repeat_count**: Specifies how many times the LCA should execute the query and generate links. A value of `0` makes the process run indefinitely.  
* **query_interval**: Defines the delay (in seconds) between consecutive request cycle, allowing control over how frequently new links are generated.  
* **query_timeout**: Sets the maximum duration (in seconds) allowed for each request cycle to complete. A value of `0` disables the timeout, allowing the process to run until it finishes naturally.  


In [ ]:
run('!(das-get-params!)')
run('!(das-set-param! (max_answers 10))')

## Using Templates

To generate new expressions with LCA, define a query and one or more templates. For each result produced by the query, LCA will create new expressions based on the specified templates.

For example, the query
`(and (Evaluation (Predicate "is_mammal") (Concept $C1)) (Evaluation (Predicate "is_mammal") (Concept $C2)))`
returns all pairs of animals classified as mammals.

The templates `(MyCustomRelationship $C1 $C2)` and `(MyCustomRelationship $C2 $C1)` will then generate two expressions for each identified pair of animals.

In [ ]:
run('''
    !(das-link-creation! 
        (and 
            (Evaluation (Predicate "is_mammal") (Concept $C1)) 
            (Evaluation (Predicate "is_mammal") (Concept $C2))
        ) 
        (MyCustomRelationship $C1 $C2) 
        (MyCustomRelationship $C2 $C1)
    )
    ''')

To check the expressions run:

In [ ]:
run('!(match &das (MyCustomRelationship $P1 $P2) (MyCustomRelationship $P1 $P2))')

Nested Templates Example:

LCA supports nested expressions as templates and automatically generates all corresponding instantiated expressions.  
Given the expression `(and (Evaluation (Predicate $P1) (Concept $C1)) (Evaluation (Predicate $P1) (Concept $C2)))` and the template `(MyCustomRelationship ($P1 $C1) ($P1 $C2))`,  
LCA will create expressions linking the Predicate `$P1` applied to `$C1` and `$C2` through the `MyCustomRelationship` relation.

Example:  
P1 = "is_animal"  
C1 = "rhino"  
C2 = "monkey"  

Result:  
`(MyCustomRelationship ("is_animal" "rhino") ("is_animal" "monkey"))`

In [ ]:
run('''
    !(das-link-creation! 
        (and 
            (Evaluation (Predicate $P1) (Concept $C1)) 
            (Evaluation (Predicate $P1) (Concept $C2))
        ) 
        (MyCustomRelationship ($P1 $C1) ($P1 $C2))
    )
    ''')

To check the created expressions run:

In [ ]:
run('!(match &das (MyCustomRelationship ($P1 $C1) ($P1 $C2)) (MyCustomRelationship ($P1 $C1) ($P1 $C2)))')

## Using Processors

To create expressions with an available processor. Processors take each result produced by a query and generate new expressions according to the processor’s internal logic.

Processors are useful because they can perform **specialized computations**, such as assigning additional properties to the newly created expressions.  
For example, the **IMPLICATION_RELATION** processor calculates the implication strength between two answers and stores this value as a custom property of the generated link.  
Each processor is implemented internally within LCA and encapsulates a specific algorithm or heuristic that determines how the new expressions are generated.    
New processors can be added under demand, check the current status of requests [here](https://github.com/orgs/singnet/projects/7). In future versions, LCA will also support user-defined processors implemented directly in MeTTa, allowing advanced users to prototype and integrate custom processing logic without modifying the core system.

When invoking a processor, the caller must provide a query and a unique "tag" that identifies which processor should be executed for a given query.


The available processors tags are:

* IMPLICATION_RELATION

Given a query, this processor computes the strength of implication between pairs of answers and generates new expressions representing directional relationships:
If the computed strength is greater than zero, it generates new expressions and stores the strength as an additional property of the link.  
The expressions take the form:  
`(Implication A B)` and `(Implication B A)`  
which correspond to:  
A → B and B → A
* EQUIVALENCE_RELATION

Given a query, this processor computes the strength of equivalence between pairs of answers.  
If the computed strength is greater than zero, it generates expressions and stores the strength as an additional property of the link.  
The expressions take the form:  
`(Equivalence A B)` and `(Equivalence B A)`  
which correspond to:  
A ↔ B

Processor **EQUIVALENCE_RELATION** Example:

In [ ]:
run('!(das-link-creation! (and (Concept $C1) (Concept $C2)) EQUIVALENCE_RELATION)')

To check the expressions run:

In [ ]:
run('!(match &das (Equivalence (Concept $C1) (Concept $C2)) (Equivalence (Concept $C2) (Concept $C1)))')